# SoS Singularity Guide

Before you run any script using singularity, please check if `singularity` is installed by checking the availability of command `singularity` (`which singularity`). Also, it would be helpful for you to read the [sigularity user's guide](https://www.sylabs.io/docs/) or some online tutorial to understand how singularity works before you try to use singularity in SoS.

Although running singularity images does not need root privilege, building singularity images often requires `sudo` access. SoS provides an option `sudo=True` to the `singularity_build` action (an equivalence to command `singularity build`) to execute the command in sudo mode, but it does not accept interactive input of password. So before running any `singularity_build` action with option `sudo=True`, please run `sudo -i` to enter sudo mode, or add your username as a sudo user without password (google "sudo without password" for instructions). The latter is not safe but can be very convenient if you are singularity on a personal linux workstation.

In [1]:
%revisions

Revision,Author,Date,Message
64ebdc5,Bo Peng,2018-09-08,Update toc
89058f0,Bo Peng,2018-09-08,Add singularity guide


## Running a script using singularity

SoS executes scripts inside a singularity by calling command `singularity exec` with appropriate parameters. For example, if you specify a container with a `shub:` schema, sos will first pull the image, save it as a local image, and use `singularity exec` to run it:

In [2]:
%run
run: container='shub://singularityhub/ubuntu'
  cat /etc/os-release

Pulling image shub://singularityhub/ubuntu


cat /etc/os-release
NAME="Ubuntu"
VERSION="14.04, Trusty Tahr"
ID=ubuntu
ID_LIKE=debian
PRETTY_NAME="Ubuntu 14.04 LTS"
VERSION_ID="14.04"
HOME_URL="http://www.ubuntu.com/"
SUPPORT_URL="http://help.ubuntu.com/"
BUG_REPORT_URL="http://bugs.launchpad.net/ubuntu/"


The actual `singularity exec` command executed by SoS can be shown when you execute the script in dryrun mode (with `-n` option). In this mode, SoS would print the interpolated script (if option `expand=True` is set) and the docker command to execute it:

In [3]:
%rerun -n

singularity exec singularityhub-ubuntu.simg /bin/bash -ev /home/bpeng1/sos/sos-docs/src/tutorials/tmpsr3e7w2y/singularity_run_107345.sh


cat /etc/os-release



As you can see, the docker command looks similar to

```
singularity exec singularityhub-ubuntu.simg /bin/bash -ev /home/bpeng1/sos/sos-docs/src/tutorials/tmpr76ii92q/singularity_run_53574.sh
```

Basically, SoS pulls the image and runs command `singularity exec` to execte the script with `/bin/bash` inside the image.

You can also use a docker image. However, because `docker://` images are by default executed by docker, you will need to specify the use of singularity using parameter `engine='singularity'`:

In [4]:
%run
run: container='docker://ubuntu', engine='singularity'
  cat /etc/os-release

cat /etc/os-release
NAME="Ubuntu"
VERSION="18.04.1 LTS (Bionic Beaver)"
ID=ubuntu
ID_LIKE=debian
PRETTY_NAME="Ubuntu 18.04.1 LTS"
VERSION_ID="18.04"
HOME_URL="https://www.ubuntu.com/"
SUPPORT_URL="https://help.ubuntu.com/"
BUG_REPORT_URL="https://bugs.launchpad.net/ubuntu/"
PRIVACY_POLICY_URL="https://www.ubuntu.com/legal/terms-and-policies/privacy-policy"
VERSION_CODENAME=bionic
UBUNTU_CODENAME=bionic


In summary, as listed [here](https://vatlab.github.io/sos-docs/doc/documentation/Targets_and_Actions.html#container-and-engine-124), you can invoke `singularity` with the following combinations of parameter `container` and `engine`:

| `container` | `engine` | execute by | example | comment | 
| -- | -- | -- | -- | -- |
| `filename.simg` | ` ` | singularity | `container='ubuntu.simg'` | |
| `shub://tag` | ` ` | singularity | `container='shub://GodloveD/lolcow'` | Image will be pulled to a local image |
| `name` | `singularity` | singularity | `container='a_dir', engine='singularity'` | treat `name` as singularity image file or directory |
| `docker://tag` | `singularity` | singularity |  `container='docker://godlovdc/lolcow', engine='singularity'`  |  |
| `file://filename` | ` ` | singularity | `container='file://ubuntu.simg'` | |

Simply put, singularity will be used by default with container `shub://`, `file://`, `filename.simg`, but you will have to specify `engine='singularity'` if you would like to use a docker image.

## Building singularity images

### Action `singularity_build` <a id="singularity_build"></a>

Action `singularity_build` calls command `singularity build` with appropriate command line options. The SoS equivalence of the sample command

```
singularity build lolcow.simg shub://GodloveD/lolcow
```
is the `singularity_build` action with options `src` and `dest`:

In [5]:
singularity_build(src='shub://GodloveD/lolcow', dest='lolcow.simg')

Cache folder set to /home/bpeng1/.singularity/shub
Progress |===================================| 100.0% 
cat: write error: Broken pipe
Building from local image: /home/bpeng1/.singularity/shub/GodloveD-lolcow-master-latest.simg
(BWARNING: it may be missing some functionality.
(BBuilding Singularity image...
Singularity container built: lolcow.simg
Cleaning up...


0

If you need to build the image with root privilege, you can add option `sudo=True` when you are certain that the user is in SUDO (no password) mode:

In [6]:
singularity_build(src='shub://GodloveD/lolcow', dest='lolcow_sudo.simg', sudo=True)

Cache folder set to /root/.singularity/shub
Progress |===================================| 100.0% 
cat: write error: Broken pipe
Building from local image: /root/.singularity/shub/GodloveD-lolcow-master-latest.simg
Building Singularity image...
Singularity container built: lolcow_sudo.simg
Cleaning up...


0

You can also use action `singularity_build` to build an image from a singularity definition file. Using the example from the user's guide:

In [7]:
singularity_build: dest='lolcow.simg', sudo=True
    Bootstrap: docker
    From: ubuntu:16.04
    %post
        apt-get -y update
        apt-get -y install fortune cowsay lolcat
    %environment
        export LC_ALL=C
        export PATH=/usr/games:$PATH
    %runscript
        fortune | cowsay | lolcat 

Building into existing container: lolcow.simg
Using container recipe deffile: /home/bpeng1/sos/sos-docs/src/tutorials/tmpuvnehg53/singularity.def
Sanitizing environment
Adding base Singularity environment to container
User defined %runscript found! Taking priority.
Adding environment to container
Running post scriptlet
+ apt-get -y update
Get:1 http://security.ubuntu.com/ubuntu xenial-security InRelease [107 kB]
Hit:2 http://archive.ubuntu.com/ubuntu xenial InRelease
Get:3 http://archive.ubuntu.com/ubuntu xenial-updates InRelease [109 kB]       
Get:4 http://security.ubuntu.com/ubuntu xenial-security/universe Sources [90.3 kB]
Get:5 http://archive.ubuntu.com/ubuntu xenial-backports InRelease [107 kB]     
Get:6 http://security.ubuntu.com/ubuntu xenial-security/main amd64 Packages [703 kB]
Get:7 http://archive.ubuntu.com/ubuntu xenial-updates/universe Sources [277 kB]
Get:8 http://security.ubuntu.com/ubuntu xenial-security/restricted amd64 Packages [12.7 kB]
Get:9 http://security.ubuntu

0

Options such as `notest=True` could be add to the action. Note that the content of the definition file is indented for  clarify, but you can include the file as it is (no indentation).

## Running script with singularity image (`singularity exec`)

Although singularity accepts `docker://` and `shub://` container tags, SoS always pull the image and build a local `.simg` file before executing it. If the container is used again, the local `simg` file will be used directly.

For example, the following action will pull `docker://ubuntu` and create `ubuntu.simg`

In [8]:
run: container='docker://ubuntu', engine='singularity'
  ls /

ls /
bin   dev	   etc	 lib	media  opt   root  sbin		srv  tmp  var
boot  environment  home  lib64	mnt    proc  run   singularity	sys  usr


and you can use `container='ubuntu.simg'` directly if you have an exiting `ubuntu.simg` file

In [9]:
run: container='ubuntu.simg'
  ls /

ls /
bin   dev	   etc	 lib	media  opt   root  sbin		srv  tmp  var
boot  environment  home  lib64	mnt    proc  run   singularity	sys  usr


### Binding directories (option `bind`)

An interesting fact about singularity is that you can use the container almost as a local command with access to local file system,

In [10]:
run: container='ubuntu.simg'
  wc -l ~/.bashrc

wc -l ~/.bashrc
128 /home/bpeng1/.bashrc


However, singularity only binds current, `/tmp`, and home directories. Other directories would be from within the image, or appear to be missing even if they exist on the host file system. For example, the following command lists `/usr/local` inside the image:

In [11]:
run: container='ubuntu.simg'
  ls /usr/local

ls /usr/local
bin  etc  games  include  lib  man  sbin  share  src


and `/usr/local/var` would appear to be missing

In [12]:
run: container='ubuntu.simg'
  ls /usr/local/var

ls /usr/local/var
ls: cannot access '/usr/local/var': No such file or directory


Executing script in Singularity returns an error (exitcode=2).
The script has been saved to /home/bpeng1/sos/sos-docs/src/tutorials/.sos/singularity_run_107345.sh. To reproduce the error please run:
``singularity exec  ubuntu.simg /bin/bash -ev /home/bpeng1/sos/sos-docs/src/tutorials/.sos/singularity_run_107345.sh``


To allow singularity to see more directories, you can add one or more parameters to the `bind` parameter. For example, with `bind='/usr/local'`, the `singularity exec` command lists directory `/usr/local` from the host filesystem:

In [13]:
run: container='ubuntu.simg', bind='/usr/local'
  ls /usr/local

ls /usr/local
bin  etc  games  include  lib  libexec	man  sbin  share  src  var


and we can see `/usr/local/var` actually exists

In [14]:
run: container='ubuntu.simg', bind='/usr/local'
  ls /usr/local/var

ls /usr/local/var
singularity


Parameter `bind` accepts `host_dir:img_dir` pairs to mount `host_dir` from host as `img_dir` seen by the image

In [15]:
%run -v3
run: container='ubuntu.simg', bind='/usr/local/var:/myvar'
  ls /myvar

ls /myvar
singularity
